# 영화진흥위원회 데이터 수집

## setting

In [9]:
import requests

class URLMaker:
    """
    영화진흥위원회 API URL 생성 클래스
    """
    
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest'

    def __init__(self, key):
        """
        API key 초기화
        """
        self.key = key

    def daily_boxoffice_url(self, targetDt, itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''): # 기본값으로 넣어주면 옵션으로 필수적으로 넣지 않아도 동작
        """
        일별 박스오피스 URL 생성
        """
        return f'{self.url}/boxoffice/searchDailyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'

    def weekly_boxoffice_url(self, targetDt, weekGb='0', itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        주간/주말 박스오피스 URL 생성
        weekGb: 0(주간), 1(주말), 2(주중)
        """
        return f'{self.url}/boxoffice/searchWeeklyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&weekGb={weekGb}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'

    def movie_list_url(self, curPage=1, itemPerPage=10, movieNm='', directorNm='', openStartDt='', openEndDt='', 
                      prdtStartYear='', prdtEndYear='', repNationCd='', movieTypeCd=''):
        """
        영화목록 URL 생성
        """
        return f'{self.url}/movie/searchMovieList.json?key={self.key}&curPage={curPage}&itemPerPage={itemPerPage}&movieNm={movieNm}&directorNm={directorNm}&openStartDt={openStartDt}&openEndDt={openEndDt}&prdtStartYear={prdtStartYear}&prdtEndYear={prdtEndYear}&repNationCd={repNationCd}&movieTypeCd={movieTypeCd}'

    def movie_info_url(self, movieCd):
        """
        영화 상세정보 URL 생성
        """
        return f'{self.url}/movie/searchMovieInfo.json?key={self.key}&movieCd={movieCd}'

    def people_list_url(self, curPage=1, itemPerPage=10, peopleNm='', filmoNames=''):
        """
        영화인목록 URL 생성
        """
        return f'{self.url}/people/searchPeopleList.json?key={self.key}&curPage={curPage}&itemPerPage={itemPerPage}&peopleNm={peopleNm}&filmoNames={filmoNames}'

    def people_info_url(self, peopleCd):
        """
        영화인 상세정보 URL 생성
        """
        return f'{self.url}/people/searchPeopleInfo.json?key={self.key}&peopleCd={peopleCd}'


    

In [10]:
MY_API_KEY = '6d114e0572cd2106801b8160c3cfbfe5'
url_maker = URLMaker(MY_API_KEY) #url_maker 인스턴스 생성

In [11]:
url_maker.key

'6d114e0572cd2106801b8160c3cfbfe5'

In [36]:
# url_maker 인스턴스 안에서
# daily_url, weekly_url, info_url 정의 = 모계 안의 정보로 데일리박스오피스 url생성
daily_url = url_maker.daily_boxoffice_url('20240201')
weekly_url = url_maker.weekly_boxoffice_url('20250201')
info_url = url_maker.movie_info_url('19950084')
print(info_url)

http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=6d114e0572cd2106801b8160c3cfbfe5&movieCd=19950084


In [37]:
import requests
info = requests.get(info_url)
info_dict = info.json()

movie_name = info_dict[f'movieInfoResult'][f'movieInfo'][f'movieNm']
movie_years = info_dict[f'movieInfoResult'][f'movieInfo'][f'prdtYear']
movie_dirc = info_dict[f'movieInfoResult'][f'movieInfo'][f'directors'][0][f'peopleNm']
movie_act1 = info_dict[f'movieInfoResult'][f'movieInfo'][f'actors'][0][f'peopleNm']
movie_act2 = info_dict[f'movieInfoResult'][f'movieInfo'][f'actors'][1][f'peopleNm']
movie_gen = info_dict[f'movieInfoResult'][f'movieInfo'][f'genres'][0][f'genreNm']

## 영화 상세정보 조회

- **영화 상세정보 api**를 사용합니다.
- 영화 코드(movieCd)를 입력받아 해당 영화의 상세 정보를 조회합니다.
- 다음 정보를 순서대로 출력합니다:
    - 영화 제목 (제작연도)
    - 감독 (첫 번째 감독)
    - 주연 배우 (첫 번째, 두 번째 배우)
    - 장르 (첫 번째 장르)

In [14]:
def get_movie_detail(movie_cd):
    print(f'{movie_name} ({movie_years})')
    print(f'감독 : {movie_dirc}')
    print(f'주연 : {movie_act1}, {movie_act2}')
    print(f'장르 : {movie_gen}')

print(get_movie_detail(19950084))

쇼생크 탈출 (1994)
감독 : 프랭크 다라본트
주연 : 팀 로빈스, 모건 프리먼
장르 : 드라마
None


In [15]:
get_movie_detail('19950084')

쇼생크 탈출 (1994)
감독 : 프랭크 다라본트
주연 : 팀 로빈스, 모건 프리먼
장르 : 드라마


**출력 예시**
```
쇼생크 탈출 (1994)
감독: 프랭크 다라본트
주연: 팀 로빈스, 모건 프리먼
장르: 드라마
```

## 일별 박스오피스 조회

- **일별 박스오피스 api**를 사용합니다.
- 입력받은 날짜(YYYYMMDD 형식)의 박스오피스 정보를 조회합니다.
- 각 영화의 순위, 영화제목, 해당 일의 관객수를 출력합니다.

In [16]:
daily_url = url_maker.daily_boxoffice_url('20250201')
print(daily_url)

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=6d114e0572cd2106801b8160c3cfbfe5&targetDt=20250201&itemPerPage=10&multiMovieYn=&repNationCd=&wideAreaCd=


In [17]:
import requests
daily = requests.get(daily_url)
daily_dict = daily.json()

In [18]:
daily2 = daily_dict[f'boxOfficeResult'][f'dailyBoxOfficeList']
# daily2 = daily_dict의 리스트1 안의 리스트2에 접근

def get_daily_boxoffice(target_date):
    print(f'{target_date} 박스오피스 TOP 10')
    for i in range(10): # 0부터 9까지 (총 10개)의 순위만큼 뽑기
        print(f'{daily2[i][f'rnum']}위 : {daily2[i][f'movieNm']}')
        # 리스트 2의 i번째 딕셔너리의 [rnum]의 value를 불러오기:
        # 리스트 2의 i번째 딕셔너리의 [movieNm]의 value를 불러오기
        
print(get_daily_boxoffice("20250201"))


20250201 박스오피스 TOP 10
1위 : 히트맨2
2위 : 검은 수녀들
3위 : 말할 수 없는 비밀
4위 : 하얼빈
5위 : 서브스턴스
6위 : 극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오
7위 : 뽀로로 극장판 바닷속 대모험
8위 : 바다 탐험대 옥토넛 어보브 앤 비욘드 : 극지방 대작전
9위 : 극장판 짱구는 못말려: 우리들의 공룡일기
10위 : 아이유 콘서트 : 더 위닝
None


In [19]:
get_daily_boxoffice("20250201")  # YYYYMMDD 형식으로 날짜 입력

20250201 박스오피스 TOP 10
1위 : 히트맨2
2위 : 검은 수녀들
3위 : 말할 수 없는 비밀
4위 : 하얼빈
5위 : 서브스턴스
6위 : 극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오
7위 : 뽀로로 극장판 바닷속 대모험
8위 : 바다 탐험대 옥토넛 어보브 앤 비욘드 : 극지방 대작전
9위 : 극장판 짱구는 못말려: 우리들의 공룡일기
10위 : 아이유 콘서트 : 더 위닝


**출력예시**
```
20250201 박스오피스 TOP 10
1위: 히트맨2 (177129명)
2위: 검은 수녀들 (96043명)
3위: 말할 수 없는 비밀 (63000명)
4위: 하얼빈 (23298명)
5위: 서브스턴스 (15031명)
6위: 극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오 (12342명)
7위: 뽀로로 극장판 바닷속 대모험 (5903명)
8위: 바다 탐험대 옥토넛 어보브 앤 비욘드 : 극지방 대작전 (4570명)
9위: 극장판 짱구는 못말려: 우리들의 공룡일기 (3801명)
10위: 아이유 콘서트 : 더 위닝 (3156명)
```

## 한국 영화 일별 박스오피스 조회

- **일별 박스오피스 api**를 사용합니다.
- 입력받은 날짜(YYYYMMDD 형식)의 한국 영화 박스오피스 정보를 조회합니다.
- 한국 영화만 조회하기 위해 repNationCd='K' 파라미터를 사용합니다.
- 영화가 개봉한 순서대로 정렬하여 제목, 날짜, 순위를 출력합니다.


In [20]:
daily_url = url_maker.daily_boxoffice_url('20250201', repNationCd='K')
print(daily_url)

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=6d114e0572cd2106801b8160c3cfbfe5&targetDt=20250201&itemPerPage=10&multiMovieYn=&repNationCd=K&wideAreaCd=


In [21]:
import requests
daily = requests.get(daily_url)
daily_dict = daily.json()
print(daily_dict)

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스', 'showRange': '20250201~20250201', 'dailyBoxOfficeList': [{'rnum': '1', 'rank': '1', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20233006', 'movieNm': '히트맨2', 'openDt': '2025-01-22', 'salesAmt': '1663433100', 'salesShare': '46.0', 'salesInten': '525167660', 'salesChange': '46.1', 'salesAcc': '17240257840', 'audiCnt': '177129', 'audiInten': '40686', 'audiChange': '29.8', 'audiAcc': '1831557', 'scrnCnt': '1495', 'showCnt': '6165'}, {'rnum': '2', 'rank': '2', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20247450', 'movieNm': '검은 수녀들', 'openDt': '2025-01-24', 'salesAmt': '934872220', 'salesShare': '25.8', 'salesInten': '247202440', 'salesChange': '35.9', 'salesAcc': '13206216740', 'audiCnt': '96043', 'audiInten': '14734', 'audiChange': '18.1', 'audiAcc': '1363840', 'scrnCnt': '1192', 'showCnt': '4387'}, {'rnum': '3', 'rank': '3', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20223819', 'movieNm': '말할 수 없는 비밀', 'ope

In [22]:
# # 선생님 수정코드
# movie_list = daily_dict['boxOfficeResult']['dailyBoxOfficeList']
# for movie in movie_list:
#     print(movie['movieNm'])    
#     print(movie['openDt'])

In [23]:

movie_list = daily_dict[f'boxOfficeResult'][f'dailyBoxOfficeList']
date = {}
def get_movie_details(target_date):
    i = 0
    while i < 10:
        movie = daily2[i]["movieNm"]
        movie_value = []
        open_date = daily2[i]["openDt"]
        movie_rank = daily2[i]["rank"]
        movie_value.append(open_date)
        movie_value.append(movie_rank)
        # print(movie)
        date[movie] = movie_value # date라는 빈 딕셔너리에 movie를 키로, open_date를 값으로 줌
        i = i + 1
    sorted_date = sorted(date.items(), key=lambda x: x[1])
    # for k,v in sorted_date.items():
    
    for sort in sorted_date:
        print(f'{sort[0]}: ({sort[1][0]}) - {sort[1][1]}위')

In [24]:
get_movie_details('20250201')

서브스턴스: (2024-12-11) - 5위
극장판 짱구는 못말려: 우리들의 공룡일기: (2024-12-18) - 9위
하얼빈: (2024-12-24) - 4위
뽀로로 극장판 바닷속 대모험: (2025-01-01) - 7위
히트맨2: (2025-01-22) - 1위
극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오: (2025-01-22) - 6위
아이유 콘서트 : 더 위닝: (2025-01-24) - 10위
검은 수녀들: (2025-01-24) - 2위
말할 수 없는 비밀: (2025-01-27) - 3위
바다 탐험대 옥토넛 어보브 앤 비욘드 : 극지방 대작전: (2025-01-27) - 8위


**출력예시**
```
20250201 한국 영화 박스오피스 (개봉일 순)
뜻밖의 순간 : 언익스펙티드 저니 ( ) - 10위
마당을 나온 암탉 (2011-07-28) - 9위
소방관 (2024-12-04) - 8위
하얼빈 (2024-12-24) - 4위
뽀로로 극장판 바닷속 대모험 (2025-01-01) - 5위
히트맨2 (2025-01-22) - 1위
검은 수녀들 (2025-01-24) - 2위
아이유 콘서트 : 더 위닝 (2025-01-24) - 6위
귀신경찰 (2025-01-24) - 7위
말할 수 없는 비밀 (2025-01-27) - 3위
```

## 1월 전체 주간 박스오피스 데이터 수집

- **주간/주말 박스오피스 api**를 사용합니다.
- 2025년 1월의 모든 주간(1/5, 1/12, 1/19, 1/26) 데이터를 수집합니다.
- 각 영화의 정보는 다음을 포함해야 합니다:
    - 영화제목, 개봉일, 누적관객수, 누적매출액
- 동일한 영화가 여러 주에 걸쳐 나타날 경우, 가장 최근의 정보만 저장합니다.


In [25]:
weekly_0105_url = url_maker.weekly_boxoffice_url('20250105')
weekly_0112_url = url_maker.weekly_boxoffice_url('20250112')
weekly_0119_url = url_maker.weekly_boxoffice_url('20250119')
weekly_0126_url = url_maker.weekly_boxoffice_url('20250126')

print(weekly_0105_url)
print(weekly_0112_url)
print(weekly_0119_url)
print(weekly_0126_url)


http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=6d114e0572cd2106801b8160c3cfbfe5&targetDt=20250105&weekGb=0&itemPerPage=10&multiMovieYn=&repNationCd=&wideAreaCd=
http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=6d114e0572cd2106801b8160c3cfbfe5&targetDt=20250112&weekGb=0&itemPerPage=10&multiMovieYn=&repNationCd=&wideAreaCd=
http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=6d114e0572cd2106801b8160c3cfbfe5&targetDt=20250119&weekGb=0&itemPerPage=10&multiMovieYn=&repNationCd=&wideAreaCd=
http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchWeeklyBoxOfficeList.json?key=6d114e0572cd2106801b8160c3cfbfe5&targetDt=20250126&weekGb=0&itemPerPage=10&multiMovieYn=&repNationCd=&wideAreaCd=


In [26]:
import requests
weekly_0126 = requests.get(weekly_0126_url)
weekly_0126_dict = weekly_0126.json()
import requests
weekly_0119 = requests.get(weekly_0119_url)
weekly_0119_dict = weekly_0119.json()
import requests
weekly_0112 = requests.get(weekly_0112_url)
weekly_0112_dict = weekly_0112.json()
import requests
weekly_0105 = requests.get(weekly_0105_url)
weekly_0105_dict = weekly_0105.json()

In [33]:
weekly_0126_list = weekly_0126_dict[f'boxOfficeResult'][f'weeklyBoxOfficeList']
weekly_0119_list = weekly_0119_dict[f'boxOfficeResult'][f'weeklyBoxOfficeList']
weekly_0112_list = weekly_0126_dict[f'boxOfficeResult'][f'weeklyBoxOfficeList']
weekly_0105_list = weekly_0119_dict[f'boxOfficeResult'][f'weeklyBoxOfficeList']
box_data_append = {}

for i in range(0,len(weekly_0126_list)):
    box_data_i = {
    '영화제목': '',
    '개봉일': '',
    '누적관객': '',
    '누적매출': '',
}
    box_data_i['영화제목'] = weekly_0126_list[i]["movieNm"]
    box_data_i['개봉일'] = weekly_0126_list[i]["openDt"]
    box_data_i['누적관객'] = weekly_0126_list[i]["audiAcc"]
    box_data_i['누적매출'] = weekly_0126_list[i]["salesAcc"]
    box_data_append[weekly_0126_list[i]["movieNm"]] = box_data_i

## 0126 영화이름
movie_name_26 = []
for movie in weekly_0126_list:
    movie_name_26.append(movie['movieNm'])



for i in range(0,len(weekly_0119_list)):
    box_data_i = {
    '영화제목': '',
    '개봉일': '',
    '누적관객': '',
    '누적매출': '',
}
    if weekly_0119_list[i]["movieNm"] in movie_name_26:
        continue
    else:
        box_data_i['영화제목'] = weekly_0119_list[i]["movieNm"]
        box_data_i['개봉일'] = weekly_0119_list[i]["openDt"]
        box_data_i['누적관객'] = weekly_0119_list[i]["audiAcc"]
        box_data_i['누적매출'] = weekly_0119_list[i]["salesAcc"]
        box_data_append[weekly_0119_list[i]["movieNm"]] = box_data_i


# 0119 영화이름
movie_name_19 = []
for movie in weekly_0119_list:
    movie_name_19.append(movie['movieNm'])
    
for i in range(0,len(weekly_0112_list)):
    box_data_i = {
    '영화제목': '',
    '개봉일': '',
    '누적관객': '',
    '누적매출': '',
}
    if weekly_0112_list[i]["movieNm"] in movie_name_19:
        continue
    else:
        box_data_i['영화제목'] = weekly_0112_list[i]["movieNm"]
        box_data_i['개봉일'] = weekly_0112_list[i]["openDt"]
        box_data_i['누적관객'] = weekly_0112_list[i]["audiAcc"]
        box_data_i['누적매출'] = weekly_0112_list[i]["salesAcc"]
        box_data_append[weekly_0112_list[i]["movieNm"]] = box_data_i


#0112 영화이름
movie_name_12 = []
for movie in weekly_0112_list:
    movie_name_12.append(movie['movieNm'])
    

for i in range(0,len(weekly_0105_list)):
    box_data_i = {
    '영화제목': '',
    '개봉일': '',
    '누적관객': '',
    '누적매출': '',
}
    if weekly_0105_list[i]["movieNm"] in movie_name_12:
        continue
    else:
        box_data_i['영화제목'] = weekly_0105_list[i]["movieNm"]
        box_data_i['개봉일'] = weekly_0105_list[i]["openDt"]
        box_data_i['누적관객'] = weekly_0105_list[i]["audiAcc"]
        box_data_i['누적매출'] = weekly_0105_list[i]["salesAcc"]
        box_data_append[weekly_0105_list[i]["movieNm"]] = box_data_i
print(box_data_append)



{'동화지만 청불입니다': {'영화제목': '동화지만 청불입니다', '개봉일': '2025-01-08', '누적관객': '157616', '누적매출': '1465916720'}, '수퍼 소닉3': {'영화제목': '수퍼 소닉3', '개봉일': '2025-01-01', '누적관객': '322100', '누적매출': '2950620910'}, '위키드': {'영화제목': '위키드', '개봉일': '2024-11-20', '누적관객': '2195165', '누적매출': '21677201940'}, '모아나 2': {'영화제목': '모아나 2', '개봉일': '2024-11-27', '누적관객': '3522517', '누적매출': '32533317750'}, '무파사: 라이온 킹': {'영화제목': '무파사: 라이온 킹', '개봉일': '2024-12-18', '누적관객': '833832', '누적매출': '8188072950'}, '검은 수녀들': {'영화제목': '검은 수녀들', '개봉일': '2025-01-24', '누적관객': '606145', '누적매출': '5870106460'}, '히트맨2': {'영화제목': '히트맨2', '개봉일': '2025-01-22', '누적관객': '604875', '누적매출': '5686993120'}, '극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오': {'영화제목': '극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오', '개봉일': '2025-01-22', '누적관객': '64585', '누적매출': '570033160'}, '아이유 콘서트 : 더 위닝': {'영화제목': '아이유 콘서트 : 더 위닝', '개봉일': '2025-01-24', '누적관객': '28061', '누적매출': '778067000'}, '귀신경찰': {'영화제목': '귀신경찰', '개봉일': '2025-01-24', '누적관객': '21949', '누적매출': '191035980'}}
None


In [319]:
def get_weekly_boxoffice():
    pass

In [ ]:
get_weekly_boxoffice()

**출력예시**
```
{'하얼빈': {'영화제목': '하얼빈', '개봉일': '2024-12-24', '누적관객': '4642215', '누적매출': '44407362460'}, '소방관': {'영화제목': '소방관', '개봉일': '2024-12-04', '누적관객': '3840551', '누적매출': '36145572270'}, '보고타: 마지막 기회의 땅': {'영화제목': '보고타: 마지막 기회의 땅', '개봉일': '2024-12-31', '누적관객': '396287', '누적매출': '3669720540'}, '수퍼 소닉3': {'영화제목': '수퍼 소닉3', '개봉일': '2025-01-01', '누적관객': '322100', '누적매출': '2950620910'}, '뽀로로 극장판 바닷속 대모험': {'영화제목': '뽀로로 극장판 바닷속 대모험', '개봉일': '2025-01-01', '누적관객': '349597', '누적매출': '2996015770'}, '무파사: 라이온 킹': {'영화제목': '무파사: 라이온 킹', '개봉일': '2024-12-18', '누적관객': '833832', '누적매출': '8188072950'}, '모아나 2': {'영화제목': '모아나 2', '개봉일': '2024-11-27', '누적관객': '3522517', '누적매출': '32533317750'}, '극장판 짱구는 못말려: 우리들의 공룡일기': {'영화제목': '극장판 짱구는 못말려: 우리들의 공룡일기', '개봉일': '2024-12-18', '누적관객': '762895', '누적매출': '7177275100'}, '위키드': {'영화제목': '위키드', '개봉일': '2024-11-20', '누적관객': '2195165', '누적매출': '21677201940'}, '시빌 워: 분열의 시대': {'영화제목': '시빌 워: 분열의 시대', '개봉일': '2024-12-31', '누적관객': '80373', '누적매출': '836845230'}, '동화지만 청불입니다': {'영화제목': '동화지만 청불입니다', '개봉일': '2025-01-08', '누적관객': '157616', '누적매출': '1465916720'}, '서브스턴스': {'영화제목': '서브스턴스', '개봉일': '2024-12-11', '누적관객': '302870', '누적매출': '2954609400'}, '검은 수녀들': {'영화제목': '검은 수녀들', '개봉일': '2025-01-24', '누적관객': '606145', '누적매출': '5870106460'}, '히트맨2': {'영화제목': '히트맨2', '개봉일': '2025-01-22', '누적관객': '604875', '누적매출': '5686993120'}, '극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오': {'영화제목': '극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오', '개봉일': '2025-01-22', '누적관객': '64585', '누적매출': '570033160'}, '아이유 콘서트 : 더 위닝': {'영화제목': '아이유 콘서트 : 더 위닝', '개봉일': '2025-01-24', '누적관객': '28061', '누적매출': '778067000'}, '귀신경찰': {'영화제목': '귀신경찰', '개봉일': '2025-01-24', '누적관객': '21949', '누적매출': '191035980'}}
```

## 일별 박스오피스 영화의 상세정보

- **일별 박스오피스 api**와 **영화 상세정보 api**를 사용합니다.
- 입력받은 날짜의 일별 박스오피스 TOP 10을 조회합니다.
- 각 영화의 상세 정보를 추가로 조회하여 다음 정보를 출력합니다:
    - 순위
    - 영화제목
    - 감독
    - 주연배우 (최대 3명)

In [ ]:
def get_movie_details(target_date):
    pass

In [ ]:
get_movie_details("20250201")  # YYYYMMDD 형식으로 날짜 입력

**출력예시**

```
1위
히트맨2
감독: 최원섭
주연: 권상우, 정준호, 이이경
----------------------------------------

2위
검은 수녀들
감독: 권혁재
주연: 송혜교, 전여빈, 이진욱
----------------------------------------

3위
말할 수 없는 비밀
감독: 서유민
주연: 도경수, 원진아, 신예은
----------------------------------------
...
```

## 배우검색

- **영화인목록 api**, **영화인 상세정보 api**, **영화 상세정보 api**를 사용합니다.
- 배우 이름을 입력받아 해당 배우의 정보를 검색합니다.
- 여러 명의 동명 배우가 있는 경우, 필모그래피가 가장 많은 배우를 선택합니다.
- 선택된 배우의 출연작 중 '배우'로 참여한 작품 5개의 정보를 출력합니다:
    - 영화 제목
    - 개봉일
    - 상영시간
    - 감독

In [ ]:
def analyze_actor_movies(actor_name):
    pass

In [ ]:
analyze_actor_movies("박정민")  # 원하는 배우 이름으로 변경 가능

**출력예시**

```
배우 '박정민'의 최근 출연작 5개

제목: 하얼빈
개봉일: 20241224
상영시간: 113분
감독: 우민호

제목: 얼굴
개봉일: 
상영시간: 분
감독: 연상호

제목: 전, 란
개봉일: 
상영시간: 126분
감독: 김상만

제목: 휴민트
개봉일: 
상영시간: 분
감독: 류승완

제목: 밀수
개봉일: 20230726
상영시간: 129분
감독: 류승완
```